In [1]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("yye090_722").getOrCreate()

In [2]:
df = spark.read.load("./final1.csv",format ="csv",header="true")

In [3]:
from pyspark.sql import *
changedTypedf = df.withColumn("year", df["year"].cast("integer")).withColumn("month", df["month"].cast("integer"))\
.withColumn("hour", df["hour"].cast("integer"))\
.withColumn("day", df["day"].cast("integer"))\
.withColumn("PM25", df["PM25"].cast("float"))\
.withColumn("SO2", df["SO2"].cast("float"))\
.withColumn("NO2", df["NO2"].cast("float"))\
.withColumn("CO", df["CO"].cast("float"))\
.withColumn("O3", df["O3"].cast("float"))\
.withColumn("TEMP", df["TEMP"].cast("float"))\
.withColumn("WSPM", df["WSPM"].cast("float"))
changedTypedf.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- WSPM: float (nullable = true)
 |-- station: string (nullable = true)
 |-- PM25: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- O3: float (nullable = true)
 |-- TEMP: float (nullable = true)
 |-- Season: string (nullable = true)
 |-- PM_Rating: string (nullable = true)



In [4]:
from pyspark.ml.feature import Normalizer, VectorAssembler, StandardScaler, StringIndexer
cat_column = ["station"]
for i in cat_column:
    indexer = StringIndexer(inputCol=i, outputCol=i + "Index")
    model = indexer.fit(df)
    new = model.transform(df)
    df = new
df.show()

+----+-----+---+----+----+------------+----+----+----+-----+----+----+------+---------+------------+
|year|month|day|hour|WSPM|     station|PM25| SO2| NO2|   CO|  O3|TEMP|Season|PM_Rating|stationIndex|
+----+-----+---+----+----+------------+----+----+----+-----+----+----+------+---------+------------+
|2013|    3|  1|   0| 5.7|Nongzhanguan| 5.0| 4.0|12.0|200.0|85.0|-0.5|Spring|Excellent|         7.0|
|2013|    3|  1|   1| 3.9|Nongzhanguan| 8.0| 6.0|14.0|200.0|84.0|-0.7|Spring|Excellent|         7.0|
|2013|    3|  1|   2| 5.3|Nongzhanguan| 3.0| 5.0|14.0|200.0|83.0|-1.2|Spring|Excellent|         7.0|
|2013|    3|  1|   3| 4.9|Nongzhanguan| 5.0| 5.0|14.0|200.0|84.0|-1.4|Spring|Excellent|         7.0|
|2013|    3|  1|   4| 3.2|Nongzhanguan| 5.0| 6.0|21.0|200.0|77.0|-1.9|Spring|Excellent|         7.0|
|2013|    3|  1|   5| 2.4|Nongzhanguan| 3.0|13.0|21.0|300.0|77.0|-2.4|Spring|Excellent|         7.0|
|2013|    3|  1|   6| 2.2|Nongzhanguan| 4.0|15.0|32.0|300.0|62.0|-2.5|Spring|Excellent|    

In [6]:
from pyspark.sql import *
new_df = df.drop(*["station","Season","PM_Rating"])
new_df = new_df.withColumn("year", new_df["year"].cast("integer")).withColumn("month", new_df["month"].cast("integer"))\
.withColumn("hour", new_df["hour"].cast("integer"))\
.withColumn("day", new_df["day"].cast("integer"))\
.withColumn("PM25", new_df["PM25"].cast("float"))\
.withColumn("SO2", new_df["SO2"].cast("float"))\
.withColumn("NO2", new_df["NO2"].cast("float"))\
.withColumn("CO", new_df["CO"].cast("float"))\
.withColumn("O3", new_df["O3"].cast("float"))\
.withColumn("TEMP", new_df["TEMP"].cast("float"))\
.withColumn("WSPM", new_df["WSPM"].cast("float"))
new_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- WSPM: float (nullable = true)
 |-- PM25: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- O3: float (nullable = true)
 |-- TEMP: float (nullable = true)
 |-- stationIndex: double (nullable = true)



In [12]:
new_df

+----+-----+---+----+----+----+----+----+-----+----+----+------------+
|year|month|day|hour|WSPM|PM25| SO2| NO2|   CO|  O3|TEMP|stationIndex|
+----+-----+---+----+----+----+----+----+-----+----+----+------------+
|2013|    3|  1|   0| 3.1| 7.0| 3.0| 2.0|100.0|91.0|-2.3|         1.0|
|2013|    3|  1|   5| 4.3| 4.0| 3.0| 3.0|200.0|85.0|-4.2|         1.0|
|2013|    3|  1|   6| 0.6| 3.0|33.0| 7.0|300.0|82.0|-5.9|         1.0|
|2013|    3|  1|   7| 3.4| 3.0|13.0|13.0|400.0|71.0|-2.7|         1.0|
|2013|    3|  1|   8| 4.6| 3.0|34.0|38.0|800.0|45.0|-1.6|         1.0|
|2013|    3|  1|  10| 2.7|19.0| 5.0| 3.0|300.0|85.0| 1.0|         1.0|
|2013|    3|  1|  12| 1.9| 3.0| 5.0| 3.0|300.0|86.0| 3.9|         1.0|
|2013|    3|  1|  13| 3.3| 3.0| 7.0| 3.0|300.0|87.0| 4.7|         1.0|
|2013|    3|  1|  14| 1.9| 3.0| 9.0| 6.0|300.0|87.0| 5.7|         1.0|
|2013|    3|  1|  15| 3.1| 3.0| 4.0| 3.0|200.0|89.0| 5.4|         1.0|
|2013|    3|  1|  16| 3.2| 3.0|16.0| 6.0|300.0|86.0| 4.6|         1.0|
|2013|

In [6]:
from pyspark.ml.feature import RFormula
f = RFormula(formula="PM25~.",featuresCol="features",labelCol="PM25")
output = f.fit(new_df).transform(new_df)
feature_selection = output.select("features","PM25")
feature_selection.show()

+--------------------+----+
|            features|PM25|
+--------------------+----+
|[2013.0,3.0,1.0,0...| 5.0|
|[2013.0,3.0,1.0,1...| 8.0|
|[2013.0,3.0,1.0,2...| 3.0|
|[2013.0,3.0,1.0,3...| 5.0|
|[2013.0,3.0,1.0,4...| 5.0|
|[2013.0,3.0,1.0,5...| 3.0|
|[2013.0,3.0,1.0,6...| 4.0|
|[2013.0,3.0,1.0,7...| 3.0|
|[2013.0,3.0,1.0,8...| 3.0|
|[2013.0,3.0,1.0,9...|11.0|
|[2013.0,3.0,1.0,1...| 3.0|
|[2013.0,3.0,1.0,1...| 3.0|
|[2013.0,3.0,1.0,1...| 3.0|
|[2013.0,3.0,1.0,1...| 3.0|
|[2013.0,3.0,1.0,1...| 6.0|
|[2013.0,3.0,1.0,1...| 5.0|
|[2013.0,3.0,1.0,1...| 5.0|
|[2013.0,3.0,1.0,1...|10.0|
|[2013.0,3.0,1.0,1...|11.0|
|[2013.0,3.0,1.0,1...|11.0|
+--------------------+----+
only showing top 20 rows



In [7]:
train_data,test_data = feature_selection.randomSplit([0.7,0.3])

In [8]:
# Let's see our training data.
train_data.describe().show()

# And our testing data.
test_data.describe().show()

+-------+-----------------+
|summary|             PM25|
+-------+-----------------+
|  count|           150376|
|   mean|58.92798917380243|
| stddev|51.61688005442644|
|    min|              2.0|
|    max|            247.0|
+-------+-----------------+

+-------+-----------------+
|summary|             PM25|
+-------+-----------------+
|  count|            64604|
|   mean|59.27802767612478|
| stddev|51.98201006861113|
|    min|              2.0|
|    max|            247.0|
+-------+-----------------+



In [9]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='PM25')

In [31]:
lrModel = lr.fit(train_data)

In [28]:
coefficients = lrModel.coefficients
cof_col = new_df.drop(*["PM25"]).columns
print("Intercept: " + str(lrModel.intercept))
for i in range(len(cof_col)):
    print(cof_col[i] + ":  " +str(coefficients[i]))


Intercept: 1278.7466147101736
year:  -0.6437035586144365
month:  -0.9705032941522183
day:  0.10276732030771647
hour:  -0.4373188321503156
WSPM:  0.10294537842719416
SO2:  0.03861075390764678
NO2:  0.3935322244396976
CO:  0.05925330203979991
O3:  0.20371076241833677
TEMP:  0.6031295884097563
stationIndex:  -0.9469968092326163


In [11]:
test_results = lrModel.evaluate(test_data)

In [12]:
# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()
# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|  0.836108922381527|
| 3.4865492186941083|
| -7.321259995236687|
| -4.689992952402463|
|-2.8617597581560403|
| -2.845554250407531|
| -8.671291073205111|
|0.22091144192586398|
| -5.266151365391124|
|-1.3458786532376052|
|-10.704280294052523|
| -3.518874871732578|
| -9.589942153592801|
|-13.890154742294271|
| 0.9292897047059796|
|  8.380239325617367|
| -7.156836729158613|
|  -8.26196851574582|
| -9.374430512309573|
| -8.604155035677422|
+-------------------+
only showing top 20 rows

RSME: 32.48611848123089


In [36]:
print("RootMeanSquaredError: " + str(test_results.rootMeanSquaredError))
print("MeanSquaredError: " + str(test_results.meanSquaredError))
print("R2: " + str(test_results.r2))

RootMeanSquaredError: 32.48611848123089
MeanSquaredError: 1055.347893976571
R2: 0.6094323827394463


In [15]:
# Now we can transform the unlabeled data.
predictions = lrModel.transform(test_data)
predictions.select("PM25","prediction").show()

+----+------------------+
|PM25|        prediction|
+----+------------------+
| 9.0| 8.163891077618473|
| 7.0|3.5134507813058917|
| 8.0|15.321259995236687|
| 9.0|13.689992952402463|
| 5.0|  7.86175975815604|
| 3.0| 5.845554250407531|
| 6.0|14.671291073205111|
| 8.0| 7.779088558074136|
| 9.0|14.266151365391124|
| 5.0| 6.345878653237605|
| 3.0|13.704280294052523|
| 3.0| 6.518874871732578|
| 3.0|12.589942153592801|
| 5.0| 18.89015474229427|
| 6.0|  5.07071029529402|
|10.0|1.6197606743826327|
| 4.0|11.156836729158613|
|10.0| 18.26196851574582|
| 3.0|12.374430512309573|
| 3.0|11.604155035677422|
+----+------------------+
only showing top 20 rows



In [ ]:
x = predictions.select("PM25","prediction").toPandas()
import matplotlib.pyplot as plt
plt.plot(x["PM25"])
plt.plot(x["prediction"])
plt.xlabel("PM2.5")
plt.ylabel("ID")
